In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
from google.colab import files
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
fileList = os.listdir('/content/drive/MyDrive/prova/dades')
csv_files = [file for file in fileList if file[-3:] == "csv"]
batch_size = 1000
csv_files=sorted(csv_files)

# Crear una lista vacía para almacenar los DataFrames de cada archivo
dataframes = []

# Leer cada archivo CSV y agregarlo a la lista de DataFrames
for archivo in csv_files:

    df = pd.read_csv('/content/drive/MyDrive/prova/dades/'+archivo)
    df=df.loc[:, ['num_docks_available', 'Llocs','station_id','hour','dayofweek','month','Wind','Rain']]
    dataframes.append(df)

# Combinar todos los DataFrames en uno solo
dataframe_final = pd.concat(dataframes, ignore_index=True)

# Mostrar el DataFrame resultante
print(dataframe_final)

In [ ]:
# Cargar los datos en lotes y realizar transformaciones
def load_and_process_data(data):

    data.dropna(subset=['num_docks_available', 'Llocs', 'station_id', 'hour', 'dayofweek', 'month', 'Wind', 'Rain'], inplace=True)
    data['porcio'] = data['num_docks_available'] / data['Llocs']
    data['festa'] = (data['dayofweek'] > 5) & (data['month'] == 8)
    data['winter'] = (data['month'] == 12) | (data['month'] == 1) | (data['month'] == 2)
    data['spring'] = (data['month'] >= 3) & (data['month'] <= 5)
    data['summer'] = (data['month'] >= 6) & (data['month'] <= 8)
    data['fall'] = (data['month'] >= 9) & (data['month'] <= 11)

    # X = data[['station_id', 'hour', 'festa', 'winter', 'spring', 'summer', 'fall', 'Rain']]
    X= data[['station_id', 'hour', 'festa', 'winter', 'spring', 'summer', 'fall', 'Rain','Wind']]

    Y = data['porcio']

    return X, Y
def load_and_process_data2(csv_file):
    data = pd.read_csv(csv_file)

    data['festa'] = (data['dayofweek'] > 5) & (data['month'] == 8)
    data['winter'] = (data['month'] == 12) | (data['month'] == 1) | (data['month'] == 2)
    data['spring'] = (data['month'] >= 3) & (data['month'] <= 5)
    data['summer'] = (data['month'] >= 6) & (data['month'] <= 8)
    data['fall'] = (data['month'] >= 9) & (data['month'] <= 11)

    X = data[['station_id', 'hour', 'festa', 'winter', 'spring', 'summer', 'fall', 'Rain']]
    X= data[['station_id', 'hour', 'festa', 'winter', 'spring', 'summer', 'fall', 'Rain','Wind']]


    return X

# Cargar y procesar los datos de entrenamiento
X_train, Y_train = load_and_process_data(dataframe_final)

categorical_features = ['station_id']
lgb_train = lgb.Dataset(X_train, label=Y_train, categorical_feature=categorical_features)
model = lgb.train({}, lgb_train)

# Realizar predicciones con el modelo entrenado
new_data = load_and_process_data2('/content/drive/MyDrive/prova/test/submission.csv')
# new_data_scaled = scaler.transform(new_data)
new_data_scaled = new_data

predictions = model.predict(new_data_scaled)
# Obtener los IDs de la columna correspondiente en new_data_scaled
ids = new_data.index.values

# Crear un DataFrame con las predicciones y los IDs
df = pd.DataFrame({'index': ids, 'percentage_docks_available': predictions})

# Definir la ruta y el nombre del archivo CSV
csv_file = 'lightgbm.csv'

# Guardar el DataFrame en el archivo CSV
df.to_csv(csv_file, index=False)
files.download('/content/'+csv_file)

In [ ]:
# Cargar los datos en lotes y realizar transformaciones
def load_and_process_data(data):

    data.dropna(subset=['num_docks_available', 'Llocs', 'station_id', 'hour', 'dayofweek', 'month', 'Wind', 'Rain'], inplace=True)
    data['porcio'] = data['num_docks_available'] / data['Llocs']
    data['festa'] = (data['dayofweek'] > 5) & (data['month'] == 8)
    data['winter'] = (data['month'] == 12) | (data['month'] == 1) | (data['month'] == 2)
    data['spring'] = (data['month'] >= 3) & (data['month'] <= 5)
    data['summer'] = (data['month'] >= 6) & (data['month'] <= 8)
    data['fall'] = (data['month'] >= 9) & (data['month'] <= 11)

    # X = data[['station_id', 'hour', 'festa', 'winter', 'spring', 'summer', 'fall', 'Rain']]
    X= data[['station_id', 'hour', 'festa', 'month','dayofweek', 'Rain','Wind']]
    Y = data['porcio']

    return X, Y

def load_and_process_data2(csv_file):
    data = pd.read_csv(csv_file)

    data['festa'] = (data['dayofweek'] > 5) & (data['month'] == 8)
    data['winter'] = (data['month'] == 12) | (data['month'] == 1) | (data['month'] == 2)
    data['spring'] = (data['month'] >= 3) & (data['month'] <= 5)
    data['summer'] = (data['month'] >= 6) & (data['month'] <= 8)
    data['fall'] = (data['month'] >= 9) & (data['month'] <= 11)

    # X = data[['station_id', 'hour', 'festa', 'winter', 'spring', 'summer', 'fall', 'Rain']]
    X= data[['station_id', 'hour', 'festa', 'month','dayofweek', 'Rain','Wind']]


    return X

# Cargar y procesar los datos de entrenamiento
X_train, Y_train = load_and_process_data(dataframe_final)

categorical_features = ['station_id', 'month','dayofweek']
lgb_train = lgb.Dataset(X_train, label=Y_train, categorical_feature=categorical_features)
model = lgb.train({}, lgb_train)

# Realizar predicciones con el modelo entrenado
new_data = load_and_process_data2('/content/drive/MyDrive/prova/test/submission.csv')
# new_data_scaled = scaler.transform(new_data)
new_data_scaled = new_data

predictions = model.predict(new_data_scaled)
# Obtener los IDs de la columna correspondiente en new_data_scaled
ids = new_data.index.values

# Crear un DataFrame con las predicciones y los IDs
df = pd.DataFrame({'index': ids, 'percentage_docks_available': predictions})

# Definir la ruta y el nombre del archivo CSV
csv_file = 'lightgbm.csv'

# Guardar el DataFrame en el archivo CSV
df.to_csv(csv_file, index=False)
files.download('/content/'+csv_file)